# Data Preprocessing Pipeline for Black Spot Alert System

This notebook:
1. Loads accident, tourism, and traffic datasets
2. Cleans and standardizes location names
3. Merges datasets by city/state
4. Creates aggregated features
5. Handles missing values
6. Exports ML-ready dataset


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


## 1. Load Datasets


In [2]:
def load_datasets():
    """Load all three datasets."""
    print("Loading datasets...")
    
    # Load accidents dataset
    accidents_df = pd.read_csv("us_accidents_10k_sample.csv")
    print(f"Accidents dataset: {accidents_df.shape}")
    
    # Load tourism dataset
    tourism_df = pd.read_csv("usa_tourism_dataset_10000.csv")
    print(f"Tourism dataset: {tourism_df.shape}")
    
    # Load traffic dataset
    traffic_df = pd.read_csv("synthetic_heavy_traffic_usa_10k.csv")
    print(f"Traffic dataset: {traffic_df.shape}")
    
    return accidents_df, tourism_df, traffic_df

# Load the datasets
accidents_df, tourism_df, traffic_df = load_datasets()


Loading datasets...


NameError: name 'pd' is not defined

In [ ]:
# Preview each dataset
print("\n=== Accidents Dataset ===")
print(accidents_df.head())
print("\n=== Tourism Dataset ===")
print(tourism_df.head())
print("\n=== Traffic Dataset ===")
print(traffic_df.head())


## 2. Helper Functions


In [ ]:
def clean_location_names(df, city_col, state_col):
    """Standardize city and state names."""
    if city_col in df.columns:
        df[city_col] = df[city_col].str.strip().str.title()
    if state_col in df.columns:
        df[state_col] = df[state_col].str.strip().str.upper()
    return df


## 3. Process Accidents Dataset


In [ ]:
def process_accidents(accidents_df):
    """Process accidents dataset to create city-level features."""
    print("\nProcessing accidents data...")
    
    # Clean location names
    accidents_df = clean_location_names(accidents_df, "City", "State")
    
    # Convert Start_Time to datetime
    accidents_df["Start_Time"] = pd.to_datetime(accidents_df["Start_Time"], errors="coerce")
    
    # Extract time features
    accidents_df["hour"] = accidents_df["Start_Time"].dt.hour
    accidents_df["month"] = accidents_df["Start_Time"].dt.month
    accidents_df["year"] = accidents_df["Start_Time"].dt.year
    accidents_df["is_night"] = (accidents_df["hour"] >= 20) | (accidents_df["hour"] <= 6)
    accidents_df["is_weekend"] = accidents_df["Start_Time"].dt.dayofweek >= 5
    
    # Aggregate by city and state
    city_accidents = accidents_df.groupby(["City", "State"]).agg({
        "ID": "count",  # Total accident count
        "Severity": ["mean", "max"],  # Average and max severity
        "is_night": "sum",  # Night accidents count
        "is_weekend": "sum",  # Weekend accidents count
        "Temperature(F)": "mean",  # Average temperature
        "Visibility(mi)": "mean",  # Average visibility
        "Wind_Speed(mph)": "mean",  # Average wind speed
        "Precipitation(in)": lambda x: (x > 0).sum(),  # Count of accidents with precipitation
    }).reset_index()
    
    # Flatten column names
    city_accidents.columns = [
        "City", "State",
        "accident_count",
        "avg_severity",
        "max_severity",
        "night_accidents",
        "weekend_accidents",
        "avg_temperature",
        "avg_visibility",
        "avg_wind_speed",
        "precipitation_accidents"
    ]
    
    # Calculate proportions
    city_accidents["night_accident_rate"] = (
        city_accidents["night_accidents"] / city_accidents["accident_count"]
    ).fillna(0)
    city_accidents["weekend_accident_rate"] = (
        city_accidents["weekend_accidents"] / city_accidents["accident_count"]
    ).fillna(0)
    city_accidents["precipitation_rate"] = (
        city_accidents["precipitation_accidents"] / city_accidents["accident_count"]
    ).fillna(0)
    
    print(f"Processed accidents: {city_accidents.shape[0]} unique cities")
    return city_accidents

# Process accidents
city_accidents = process_accidents(accidents_df)
print("\nAccidents aggregation preview:")
print(city_accidents.head())


## 4. Process Tourism Dataset


In [ ]:
def process_tourism(tourism_df):
    """Process tourism dataset to create city-level features."""
    print("\nProcessing tourism data...")
    
    # Clean location names
    tourism_df = clean_location_names(tourism_df, "city", "state")
    
    # Convert visit_date to datetime
    tourism_df["visit_date"] = pd.to_datetime(tourism_df["visit_date"], errors="coerce")
    
    # Aggregate by city and state
    city_tourism = tourism_df.groupby(["city", "state"]).agg({
        "record_id": "count",  # Total tourist visits
        "stay_nights": ["mean", "sum"],  # Average and total stay nights
        "avg_spend_usd_per_person": "mean",  # Average spending
        "satisfaction_rating_1_5": "mean",  # Average satisfaction
        "party_size": "mean",  # Average party size
    }).reset_index()
    
    # Flatten column names
    city_tourism.columns = [
        "City", "State",
        "tourist_visit_count",
        "avg_stay_nights",
        "total_stay_nights",
        "avg_spending",
        "avg_satisfaction",
        "avg_party_size"
    ]
    
    # Calculate tourist density (visits per city)
    city_tourism["tourist_density"] = city_tourism["tourist_visit_count"]
    
    print(f"Processed tourism: {city_tourism.shape[0]} unique cities")
    return city_tourism

# Process tourism
city_tourism = process_tourism(tourism_df)
print("\nTourism aggregation preview:")
print(city_tourism.head())


## 5. Process Traffic Dataset


In [ ]:
def process_traffic(traffic_df):
    """Process traffic dataset to create city-level features."""
    print("\nProcessing traffic data...")
    
    # Clean location names
    traffic_df = clean_location_names(traffic_df, "City", "State")
    
    # Convert Timestamp to datetime
    traffic_df["Timestamp"] = pd.to_datetime(traffic_df["Timestamp"], errors="coerce")
    
    # Aggregate by city and state
    city_traffic = traffic_df.groupby(["City", "State"]).agg({
        "Severity": ["mean", "max", "count"],  # Traffic severity metrics
        "Traffic_Volume": "mean",  # Average traffic volume
        "Avg_Speed": "mean",  # Average speed
        "Weather_Impact": lambda x: x.value_counts().index[0] if len(x) > 0 else "Unknown",  # Most common weather
    }).reset_index()
    
    # Flatten column names
    city_traffic.columns = [
        "City", "State",
        "avg_traffic_severity",
        "max_traffic_severity",
        "traffic_incident_count",
        "avg_traffic_volume",
        "avg_speed",
        "dominant_weather"
    ]
    
    print(f"Processed traffic: {city_traffic.shape[0]} unique cities")
    return city_traffic

# Process traffic
city_traffic = process_traffic(traffic_df)
print("\nTraffic aggregation preview:")
print(city_traffic.head())


## 6. Merge All Datasets


In [ ]:
def merge_datasets(city_accidents, city_tourism, city_traffic):
    """Merge all processed datasets on City and State."""
    print("\nMerging datasets...")
    
    # Start with accidents as base (most comprehensive)
    merged_df = city_accidents.copy()
    
    # Merge tourism data
    merged_df = merged_df.merge(
        city_tourism,
        on=["City", "State"],
        how="left"
    )
    
    # Merge traffic data
    merged_df = merged_df.merge(
        city_traffic,
        on=["City", "State"],
        how="left"
    )
    
    print(f"Merged dataset shape: {merged_df.shape}")
    print(f"Unique cities: {merged_df[['City', 'State']].drop_duplicates().shape[0]}")
    
    return merged_df

# Merge datasets
merged_df = merge_datasets(city_accidents, city_tourism, city_traffic)
print("\nMerged dataset preview:")
print(merged_df.head())


## 7. Handle Missing Values


In [ ]:
def handle_missing_values(df):
    """Handle missing values in the merged dataset."""
    print("\nHandling missing values...")
    print(f"Missing values before handling:")
    print(df.isnull().sum().sum())
    
    # Fill numeric columns with 0 or mean
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col not in ["City", "State"]:
            if "count" in col.lower() or "rate" in col.lower():
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna(df[col].median())
    
    # Fill categorical columns
    categorical_cols = df.select_dtypes(include=["object"]).columns
    for col in categorical_cols:
        if col not in ["City", "State"]:
            df[col] = df[col].fillna("Unknown")
    
    print(f"Missing values after handling: {df.isnull().sum().sum()}")
    return df

# Handle missing values
merged_df = handle_missing_values(merged_df)


## 8. Save Processed Dataset


In [ ]:
# Save processed dataset
output_path = "processed_blackspot_dataset.csv"
merged_df.to_csv(output_path, index=False)
print(f"✅ Saved processed dataset to: {output_path}")
print(f"Final dataset shape: {merged_df.shape}")
print(f"\nColumn names:")
print(merged_df.columns.tolist())


## 9. Dataset Summary


In [ ]:
# Display summary statistics
print("Dataset Info:")
print(merged_df.info())
print("\n\nSummary Statistics:")
print(merged_df.describe())
